In [2]:
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
os.environ["TRANSFORMERS_NO_TF"] = "1"
from underthesea import sent_tokenize

from rag_model.model.RE.final_re import *
from rag_model.model.NER.final_ner import *
from shared_functions.gg_sheet_drive import *
from shared_functions.global_functions import *
from rag_model.model.Final_pipeline.final_relation_extractor import *
from rag_model.model.Final_pipeline.final_doc_processor import *

with open('D:/Study/Education/Projects/Group_Project/rag_model/model/RE/artifact/id2relation.json', 'r') as f:
    id2relation = json.load(f)

In [3]:
# from vncorenlp import VnCoreNLP

ner_annotator = None
# ner_annotator = VnCoreNLP("D:/Study/Education/Projects/Group_Project/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g') 

ner = NER(
    model_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/model_bilstm_crf.pt",
    token2idx_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/token2idx.json",
    label2idx_path="D:/Study/Education/Projects/Group_Project/rag_model/model/NER/artifact/label2idx.json",
    annotator = ner_annotator
)

re_model = RE(checkpoint = 'D:/Study/Education/Projects/Group_Project/rag_model/model/RE/artifact/re_8_train_phobert_1_3.pth',
           use_phobert=True, id2relation=id2relation, encoder_layer=1, decoder_layer=3, use_rel_pos=False, freeze_train=True) #match the model configuration

final_re = Extractor(ner, re_model)

processor = Doc_processor(ner, re_model, final_re)

phobert = PhoBertEmbedding()

In [6]:
list_drive_files()

├── Luật
│   ├── luat_16_2023_QH15_519324.pdf
│   ├── luat_33_2024_QH15_545181.pdf
│   ├── luat_38_2019_QH14_387595.pdf
│   ├── luat_46_2024_QH15_524982.pdf
│   ├── luat_48_2024_QH15_556390.pdf
│   ├── luat_50_2024_QH15_445264.pdf
│   ├── luat_60_2024_QH15_621343.pdf
│   ├── luat_66_2025_QH15_621225.pdf
│   ├── luat_67_2025_QH15_580594.pdf
│   ├── luat_90_2025_QH15_662379.pdf
│   └── luat_97_2015_QH13_298376.pdf
├── Nghị Quyết
│   ├── nghi_quyet_01_2020_NQ-HDND_448623.pdf
│   ├── nghi_quyet_01_2025_NQ-HDND_642122.pdf
│   ├── nghi_quyet_01_2025_NQ-HDND_666755.pdf
│   ├── nghi_quyet_02_2025_NQ-HDND_650773.pdf
│   ├── nghi_quyet_04_2025_NQ-HDND_662868.pdf
│   ├── nghi_quyet_04_2025_NQ-HDND_676671.pdf
│   ├── nghi_quyet_05_2025_NQ-HDND_667873.pdf
│   ├── nghi_quyet_05_2025_NQ-HDND_675524.pdf
│   ├── nghi_quyet_05_2025_NQ-HDND_676654.pdf
│   ├── nghi_quyet_06_2025_NQ-HDND_659674.pdf
│   ├── nghi_quyet_08_2023_NQ-HDND_574278.pdf
│   ├── nghi_quyet_08_2025_NQ-HDND_655056.pdf
│   ├── nghi_quye

In [ ]:
def process_document(filename):
    doc = read_drive_file(filename)
    
    df_meta = ner.extract_document_metadata(doc)
    df_meta['filename'] = filename
    
    processor.saving_neo4j_for_retrieve(doc, 'Test_embedding')
    
    print(f'Finishing Document {filename}')
    
    write_df_to_gs(df_meta, 'File_information')